In [2]:
import os
import numpy as np
import torch
import argparse
import shutil
import time
import torch.backends.cudnn as cudnn
from tqdm import tqdm
import torch.nn.functional as F
import sys
sys.path.append('../')
import utils.losses as losses
import utils.detectors as detectors
import utils.metrics as metrics
import utils.optimizer as optim
from models.model_builder import getModel
from datasets.data_loader import getDataLoader
from torchvision import transforms as trn
from config import cfg
from PIL import Image

In [3]:
cfg = dict()
cfg['network_kind'] = 'wrn'
cfg['depth'] = 40
cfg['widen_factor'] = 2
cfg['num_classes'] = 10
cfg['drop_rate'] = 0.3
model = getModel(cfg)

In [4]:
checkpoint = torch.load("/home/sr2/Hyeokjun/OOD-saige/results/wrn_cifar10_baseline/ckpt/checkpoint_epoch_100.pyth", map_location="cpu")
model.load_state_dict(checkpoint['model_state'])

<All keys matched successfully>

In [4]:
cfg['in_dataset'] = dict()
cfg['in_dataset']['dataset'] = 'Severstal'
cfg['in_dataset']['targets'] = ['ok','1', '2']
cfg['in_dataset']['train_transform'] = trn.Compose([trn.RandomHorizontalFlip(),
                                         trn.RandomCrop(224),
                                         trn.ToTensor()])
cfg['in_dataset']['valid_transform'] = trn.Compose([trn.CenterCrop(224),
                                         trn.ToTensor()])
cfg['in_dataset']['data_root'] = '/home/sr2/HDD2/Openset/'
cfg['in_dataset']['split_root'] = '/home/sr2/Hyeokjun/OOD-saige/datasets/data_split/'

# DataLoader config
cfg['dataloader'] = dict()
cfg['dataloader']['batch_size'] = 20
cfg['dataloader']['num_workers'] = 1
cfg['dataloader']['pin_memory'] = True


in_valid_loader = getDataLoader(ds_cfg=cfg['in_dataset'],
                                    dl_cfg=cfg['dataloader'],
                                    split="valid")

Dataset Severstal ready.


In [5]:
cfg['in_dataset'] = dict()
cfg['in_dataset']['dataset'] = 'cifar10'
cfg['in_dataset']['train_transform'] = trn.Compose([trn.RandomHorizontalFlip(),
                                         trn.RandomCrop(32),
                                         trn.ToTensor()])
cfg['in_dataset']['valid_transform'] = trn.Compose([trn.CenterCrop(32),
                                         trn.ToTensor()])
cfg['in_dataset']['data_root'] = '/home/sr2/HDD2/Openset/'
cfg['in_dataset']['split_root'] = '/home/sr2/Hyeokjun/OOD-saige/datasets/data_split/'

# DataLoader config
cfg['dataloader'] = dict()
cfg['dataloader']['batch_size'] = 20
cfg['dataloader']['num_workers'] = 1
cfg['dataloader']['pin_memory'] = True


in_valid_loader = getDataLoader(ds_cfg=cfg['in_dataset'],
                                    dl_cfg=cfg['dataloader'],
                                    split="valid")

Files already downloaded and verified
Dataset CIFAR10 ready.


In [6]:
dataloader_iterator = iter(in_valid_loader)

In [9]:
(data, target) = next(dataloader_iterator)
data = data.cuda()
print(target)
logit = model(data)
print(torch.max(F.softmax(logit, dim=1),dim=1))

tensor([4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6])
torch.return_types.max(
values=tensor([0.9564, 0.9998, 0.9994, 0.9998, 0.9999, 0.9995, 1.0000, 0.9919, 0.9986,
        0.9989, 0.9999, 0.9998, 0.5427, 0.9985, 0.9982, 0.9998, 1.0000, 0.9981,
        0.5206, 0.7564], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 2, 3, 8, 8, 7, 7, 5, 3],
       device='cuda:0'))


torch.return_types.max(
values=tensor([1.0000, 0.9999, 0.9959, 0.9892, 0.9997, 0.9834, 0.7740, 0.9998, 0.9841,
        0.8313, 0.9996, 0.8667, 0.7877, 0.9940, 1.0000, 1.0000, 0.9438, 0.9982,
        0.9328, 0.9999], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0'))